In [ ]:
!pip install google-generativeai

In [ ]:
import google.generativeai as genai
from google.colab import userdata


API_KEY = userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=API_KEY)

In [ ]:
model = genai.GenerativeModel('gemini-2.0-flash')

In [ ]:
system_instruction = """Você é um assistente virtual de suporte rápido para pequenos produtores rurais.
Sua função é responder de forma clara, objetiva e prática a dúvidas comuns sobre:
- Plantio (geral)
- Pragas e doenças (identificação básica e controle inicial)
- Adubação (conceitos básicos)
- Coleta de amostras (solo, folha - princípios gerais)
- Interpretação básica de laudos simples (pH, N, P, K).
Baseie suas respostas em conhecimentos agronômicos gerais e recomende sempre buscar um profissional agrônomo para orientações, pois suas respostas são apenas para consulta.
Recomendações agronômicas e de manejo devem ser feitas por um profissional.
Se não souber a resposta ou a pergunta for muito complexa/específica, diga que a dúvida necessita de avaliação profissional no local.
Seja encorajador# Iniciar a conversa com a instrução do sistema como a primeira mensagem
chat = model.start_chat(history=[
    {
        "role": "user", # A instrução é "dada" pelo usuário
        "parts": [system_instruction]
    },
    {
        "role": "model", # O modelo "responde" (ou confirma implicitamente) a instrução
        "parts": ["Ok, compreendi. Estou pronto para ajudar."] # Uma resposta inicial opcional
    }
])

print("Chat iniciado com as instruções.") # Confirmação visual
 e use linguagem acess# --- Código anterior (Configuração, API Key, Modelo, Início do Chat) ---
# (Mantenha o código que você já executou com sucesso acima)

# --- Código para a Interação do Chatbot ---

print("\nOlá! Sou seu assistente rápido para o campo. Pergunte algo ou digite 'sair' para encerrar.")
print("--------------------------------------------------------------")

while True:
    # Pede a pergunta do usuário
    user_input = input("Você: ")

    # Verifica se o usuário quer sair
    if user_input.lower() == 'sair':
        print("Obrigado por usar o assistente! Até a próxima.")
        break # Sai do loop

    # Envia a mensagem do usuário para o modelo e obtém a resposta
    try:
        # Adiciona a mensagem do usuário ao histórico e obtém a resposta
        response = chat.send_message(user_input)

        # Exibe a resposta do assistente
        print(f"Assistente: {response.text}")

    except Exception as e:
        # Em caso de erro (ex: problema na API, conteúdo bloqueado)
        print(f"Assistente: Desculpe, não consegui processar sua pergunta no momento. ({e})")
        print("Por favor, tente perguntar de outra forma ou mais tarde.")

    print("--------------------------------------------------------------") # Separador para próxima interação
ível."""

In [ ]:
!pip install -q --upgrade gradio google-generativeai

In [ ]:
# Célula 1: Instalação/Upgrade das bibliotecas
# Usar --upgrade para garantir que estamos pegando versões mais recentes se já estiverem instaladas.
!pip install -q --upgrade gradio google-generativeai

# Célula 2: Código do Chatbot com Gradio para Colab (Corrigido)

import google.generativeai as genai
from google.colab import userdata # Para buscar a API Key no Colab
import gradio as gr

# --- Configuração da API Key no Colab ---
try:
    API_KEY = userdata.get('GOOGLE_API_KEY')
    if not API_KEY:
        print("Chave API do Google não encontrada no userdata do Colab.")
        print("Por favor, adicione sua GOOGLE_API_KEY aos Secrets do Colab (ícone de chave à esquerda).")
        raise ValueError("GOOGLE_API_KEY não configurada.")
    genai.configure(api_key=API_KEY)
except Exception as e:
    print(f"Erro ao configurar a API Key: {e}")
    raise

# --- Configuração do Modelo Generativo ---
try:
    model = genai.GenerativeModel('gemini-2.0-flash')
except Exception as e:
    print(f"Erro ao carregar o modelo GenerativeModel: {e}")
    raise

# --- Instrução do Sistema e Início do Chat ---
system_instruction = """Você é um assistente virtual de suporte rápido para pequenos produtores rurais.
Sua função é responder de forma clara, objetiva e prática a dúvidas comuns sobre:
- Plantio (geral)
- Pragas e doenças (identificação básica e controle inicial)
- Adubação (conceitos básicos)
- Coleta de amostras (solo, folha - princípios gerais)
- Interpretação básica de laudos simples (pH, N, P, K).
Baseie suas respostas em conhecimentos agronômicos gerais e recomende sempre buscar um profissional agrônomo para orientações, pois suas respostas são apenas para consulta.
Recomendações agronômicas e de manejo devem ser feitas por um profissional.
Se não souber a resposta ou a pergunta for muito complexa/específica, diga que a dúvida necessita de avaliação profissional no local.
Seja encorajador e use linguagem acessível."""

try:
    chat_global = model.start_chat(history=[
        {"role": "user", "parts": [system_instruction]},
        {"role": "model", "parts": ["Ok, compreendi. Estou pronto para ajudar os produtores rurais."]}
    ])
except Exception as e:
    print(f"Erro ao iniciar o chat com o modelo: {e}")
    raise

# --- Função de Resposta para o Gradio ---
def chatbot_response(message, history_gradio):
    """
    Função que o Gradio chamará para obter a resposta do chatbot.
    Com type='messages', history_gradio será uma lista de dicionários:
    [{'role': 'user', 'content': 'Olá'}, {'role': 'assistant', 'content': 'Oi!'}]
    Nossa implementação atual usa 'chat_global' que já mantém o histórico com o Gemini,
    então não precisamos processar 'history_gradio' diretamente para enviar ao Gemini aqui.
    """
    try:
        response = chat_global.send_message(message)
        return response.text
    except Exception as e:
        print(f"Erro ao enviar mensagem para o Gemini: {e}")
        error_details = str(e)
        if hasattr(e, 'message'):
            error_details = e.message
        elif hasattr(e, 'args') and e.args:
             error_details = str(e.args[0])

        if "block_reason" in error_details.lower():
            return "Sua pergunta não pôde ser processada devido às políticas de segurança de conteúdo. Por favor, reformule sua pergunta."
        elif "API key not valid" in error_details:
             return "A chave da API não é válida. Verifique suas credenciais."
        return f"Desculpe, ocorreu um erro ({error_details}). Por favor, tente novamente ou reformule sua pergunta."

# --- Criação da Interface com Gradio (Corrigido) ---
iface = gr.ChatInterface(
    fn=chatbot_response,
    title="Cultiva Pro (Chatbot para o Agro)",
    description="Assistente virtual para pequenos produtores rurais. Faça suas perguntas sobre plantio, pragas, adubação e mais.",
    chatbot=gr.Chatbot(height=500), # O UserWarning sobre 'type' no Chatbot é indireto; ChatInterface gerencia isso.
    textbox=gr.Textbox(placeholder="Digite sua pergunta aqui...", container=False, scale=7),
    examples=[
        ["Como faço para preparar o solo para o plantio de milho?"],
        ["Quais são os sinais de que minhas plantas estão com deficiência de nitrogênio?"],
        ["Como posso controlar pulgões de forma natural?"],
        ["Qual o pH ideal do solo para café?"]
    ],
    theme="soft",
    # Argumentos removidos devido ao TypeError: retry_btn, undo_btn, clear_btn
    # Se você atualizar o Gradio para uma versão mais recente e o erro persistir,
    # esses nomes de parâmetros podem ter mudado ou sido removidos.
    # A interface ainda terá os botões com os textos padrão.
    type='messages' # Adicionado para seguir a recomendação do UserWarning
)

# --- Lançar a Interface no Colab ---
print("Iniciando a interface do Chatbot Agro IA com Gradio no Colab...")
print("Aguarde o link público (terminado em gradio.live) aparecer abaixo.")
iface.launch(share=True, debug=True)